SreeDananjay S(21BAI1807)
Speech and Language processing Lab assignment 2

In [5]:
import pandas as pd

df = pd.read_csv("./SMSSpamCollection", sep="\t", header=None, names=["label", "message"])
print(df.head())

  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [6]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.4 MB/s eta 0:00:00a 0:00:01
Using cached click-8.1.7-py3-none-any.whl (97 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 6.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

nltk.download("stopwords")

stemmer = PorterStemmer()
stop_words = set(stopwords.words("english"))

def preprocess_text(text, use_preprocessing=True):
    text = text.lower()

    #Remove Punctuation
    text = "".join([char for char in text if char not in string.punctuation])

    if use_preprocessing:
        text = ' '.join([stemmer.stem(word) for word in text.split() if word not in stop_words])
    else:
        text = ' '.join(text.split())
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dananjay/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
df['message_processed'] = df['message'].apply(lambda x: preprocess_text(x,True))
df['message_unprocessed'] = df['message'].apply(lambda x: preprocess_text(x,False))
print(df.head())

  label                                            message  \
0   ham  Go until jurong point, crazy.. Available only ...   
1   ham                      Ok lar... Joking wif u oni...   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...   
3   ham  U dun say so early hor... U c already then say...   
4   ham  Nah I don't think he goes to usf, he lives aro...   

                                   message_processed  \
0  go jurong point crazi avail bugi n great world...   
1                              ok lar joke wif u oni   
2  free entri 2 wkli comp win fa cup final tkt 21...   
3                u dun say earli hor u c alreadi say   
4          nah dont think goe usf live around though   

                                 message_unprocessed  
0  go until jurong point crazy available only in ...  
1                            ok lar joking wif u oni  
2  free entry in 2 a wkly comp to win fa cup fina...  
3        u dun say so early hor u c already then say  
4  nah i dont t

In [9]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 649.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 5.4 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 8.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [11]:
from sklearn.model_selection import train_test_split
X_train_proc, X_test_proc, y_train_proc, y_test_proc = train_test_split(
    df['message_processed'], df['label'], test_size=0.2, random_state=42)

X_train_unproc, X_test_unproc, y_train_unproc, y_test_unproc = train_test_split(
    df['message_unprocessed'], df['label'], test_size=0.2, random_state=42)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
tfidf_vectorizer_proc = TfidfVectorizer()
X_train_proc_tfidf = tfidf_vectorizer_proc.fit_transform(X_train_proc)
X_test_proc_tfidf = tfidf_vectorizer_proc.transform(X_test_proc)

tfidf_vectorizer_unproc = TfidfVectorizer()
X_train_unproc_tfidf = tfidf_vectorizer_unproc.fit_transform(X_train_unproc)
X_test_unproc_tfidf = tfidf_vectorizer_unproc.transform(X_test_unproc)

In [18]:
from sklearn.naive_bayes import MultinomialNB

classifier_proc = MultinomialNB()
classifier_unproc = MultinomialNB()

classifier_proc.fit(X_train_proc_tfidf,y_train_proc)
y_pred_proc = classifier_proc.predict(X_test_proc_tfidf)

classifier_unproc.fit(X_train_unproc_tfidf,y_train_unproc)
y_pred_unproc = classifier_unproc.predict(X_test_unproc_tfidf)

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Evaluation function
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, pos_label='spam')
    recall = recall_score(y_true, y_pred, pos_label='spam')
    f1 = f1_score(y_true, y_pred, pos_label='spam')
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

# Evaluate with preprocessing
print("With Preprocessing:")
evaluate_model(y_test_proc, y_pred_proc)

print("Without Preprocessing")
evaluate_model(y_test_unproc, y_pred_unproc)

With Preprocessing:
Accuracy: 0.9686
Precision: 1.0000
Recall: 0.7651
F1 Score: 0.8669
Without Preprocessing
Accuracy: 0.9570
Precision: 1.0000
Recall: 0.6779
F1 Score: 0.8080


array(['008704050406', '0089my', '0121', ..., 'zouk', 'zyada', 'üll'],
      dtype=object)